In [ ]:
!cp -r /kaggle/input/autism/autism /kaggle/working/autism

In [ ]:
import cv2
import os
import pandas as pd

def is_blurry(image_path, threshold=30):
    """check if an image is blurry based on the variance of the Laplacian."""
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        return False, 0
    variance = cv2.Laplacian(image, cv2.CV_64F).var()
    return variance < threshold, variance

def find_blurry_images(root_folder, threshold=30):
    blurry_images = {"autistic": 0, "non_autistic": 0}
    blurry_list = []

    for class_name in ["autistic", "non_autistic"]:
        class_path = os.path.join(root_folder, class_name)
        for root, _, files in os.walk(class_path):
            for file in files:
                image_path = os.path.join(root, file)
                try:
                    is_blurry_flag, variance = is_blurry(image_path, threshold)
                    if is_blurry_flag:
                        blurry_images[class_name] += 1
                        blurry_list.append((image_path, variance, class_name))
                except Exception as e:
                    print(f"Error processing {image_path}: {e}")

    return blurry_images, blurry_list

train_folder = "/kaggle/working/autism/train"
threshold = 30

blurry_counts, blurry_images = find_blurry_images(train_folder, threshold)

print(f"blurry images count per class:")
for class_name, count in blurry_counts.items():
    print(f" - {class_name}: {count}")

df = pd.DataFrame(blurry_images, columns=["Image Path", "Variance", "Class"])
df.to_csv("top_blurry_images.csv", index=False)


In [ ]:
#!rm -R /kaggle/working/top_blurry_images.csv

In [ ]:
def remove_blurry_images(csv_path):
    df = pd.read_csv(csv_path)
    removed_count = 0

    for index, row in df.iterrows():
        image_path = row['Image Path']
        try:
            if os.path.exists(image_path):
                os.remove(image_path)
                print(f"Removed: {image_path}")
                removed_count += 1
            else:
                print(f"File not found: {image_path}")
        except Exception as e:
            print(f"Error removing {image_path}: {e}")

    print(f"\nTotal blurry images removed: {removed_count}")

csv_path = "/kaggle/working/top_blurry_images.csv"
remove_blurry_images(csv_path)

In [ ]:
image_path = '/kaggle/working/autism/train/non_autistic/1186.jpg'
try:
  if os.path.exists(image_path):
     os.remove(image_path)
     print(f"Removed: {image_path}")
  else:
     print(f"File not found: {image_path}")
except Exception as e:
  print(f"Error removing {image_path}: {e}")